In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from google.colab import files
import joblib
import os
import glob

🚀 Iniciando Pipeline de Machine Learning no Colab...


In [ ]:
path = "/content/"
try:
    print("Lendo CSVs...")
    orders = pd.read_csv(path + 'olist_orders_dataset.csv')
    items = pd.read_csv(path + 'olist_order_items_dataset.csv')
    customers = pd.read_csv(path + 'olist_customers_dataset.csv')
    sellers = pd.read_csv(path + 'olist_sellers_dataset.csv')
    products = pd.read_csv(path + 'olist_products_dataset.csv')
    geolocation = pd.read_csv(path + 'olist_geolocation_dataset.csv')
    print("✅ Datasets carregados!")
except FileNotFoundError as e:
    print(f"❌ Erro: Arquivos não encontrados. Verifique se fez o upload na aba esquerda.")
    print(f"Detalhe: {e}")
    # Interrompe a execução se não achar os arquivos
    raise

📂 Lendo CSVs...
✅ Datasets carregados!


In [ ]:
print("⚙️ Processando Geolocalização e Joins...")

# 3.1. Agrupar Geo (Centróide do CEP)
geo_ref = geolocation.groupby('geolocation_zip_code_prefix')[['geolocation_lat', 'geolocation_lng']].mean().reset_index()

# 3.2. Unificar Tabelas
items_first = items.drop_duplicates('order_id', keep='first')
df = orders.merge(items_first, on='order_id', how='left')
df = df.merge(customers[['customer_id', 'customer_zip_code_prefix']], on='customer_id', how='left')
df = df.merge(sellers[['seller_id', 'seller_zip_code_prefix']], on='seller_id', how='left')
df = df.merge(products[['product_id', 'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm']], on='product_id', how='left')

# 3.3. Merge das Coordenadas
# Destino (Cliente)
df = df.merge(geo_ref, left_on='customer_zip_code_prefix', right_on='geolocation_zip_code_prefix', how='left')
df.rename(columns={'geolocation_lat': 'lat_cust', 'geolocation_lng': 'lng_cust'}, inplace=True)
df.drop('geolocation_zip_code_prefix', axis=1, inplace=True)

# Origem (Vendedor)
df = df.merge(geo_ref, left_on='seller_zip_code_prefix', right_on='geolocation_zip_code_prefix', how='left')
df.rename(columns={'geolocation_lat': 'lat_sell', 'geolocation_lng': 'lng_sell'}, inplace=True)
df.drop('geolocation_zip_code_prefix', axis=1, inplace=True)

print("🧠 Calculando Distância e Features...")

# 3.4. Limpeza e Target
df = df[df['order_status'] == 'delivered'].dropna(subset=['order_delivered_customer_date', 'lat_cust', 'lat_sell', 'product_weight_g'])
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])
df['order_delivered_customer_date'] = pd.to_datetime(df['order_delivered_customer_date'])

# Target: Dias Reais
df['dias_reais'] = (df['order_delivered_customer_date'] - df['order_purchase_timestamp']).dt.days
df = df[(df['dias_reais'] > 0) & (df['dias_reais'] < 60)]

# 3.5. Feature: Distância (Haversine)
def haversine_vec(lat1, lon1, lat2, lon2):
    R = 6371
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    dphi = np.radians(lat2 - lat1)
    dlambda = np.radians(lon2 - lon1)
    a = np.sin(dphi/2)**2 + np.cos(phi1)*np.cos(phi2)*np.sin(dlambda/2)**2
    return 2*R*np.arcsin(np.sqrt(a))

df['distancia_km'] = haversine_vec(df['lat_sell'], df['lng_sell'], df['lat_cust'], df['lng_cust'])
df['volume_cm3'] = df['product_length_cm'] * df['product_height_cm'] * df['product_width_cm']

features = ['distancia_km', 'product_weight_g', 'volume_cm3', 'freight_value', 'price']
X = df[features]
y = df['dias_reais']

⚙️ Processando Geolocalização e Joins...
🧠 Calculando Distância e Features...


In [ ]:
print(f"🚀 Treinando modelo com {len(df)} registros...")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=100, max_depth=20, n_jobs=-1, random_state=42)
model.fit(X_train, y_train)

# Avaliação Rápida
mae = mean_absolute_error(y_test, model.predict(X_test))
print(f"📊 Sucesso! O modelo erra em média apenas {mae:.2f} dias.")

🚀 Treinando modelo com 95660 registros...
📊 Sucesso! O modelo erra em média apenas 5.00 dias.


In [ ]:
df_final = df[['dias_reais', 'dias_estimados_antigo', 'dias_previstos_ia']].sample(5000) # Amostra de 5k para ficar leve
df_final.to_csv('comparativo_modelo.csv', index=False)

print("✅ Arquivo 'comparativo_modelo.csv' gerado com sucesso!")

KeyError: "['dias_estimados_antigo', 'dias_previstos_ia'] not in index"

In [ ]:
print("💾 Salvando arquivos...")

# Salvar no disco do Colab
joblib.dump(model, 'modelo_entregas.joblib')
geo_ref.to_csv('referencia_geo.csv', index=False)

print("⬇️ Iniciando download para sua máquina local...")
try:
    files.download('modelo_entregas.joblib')
    files.download('referencia_geo.csv')
    print("✅ Download iniciado! Verifique sua pasta de Downloads.")
except Exception as e:
    print("⚠️ O download automático falhou (comum em alguns navegadores).")
    print("👉 Por favor, vá na aba 'Arquivos' na esquerda, clique com botão direito nos arquivos gerados e escolha 'Baixar'.")

💾 Salvando arquivos...
⬇️ Iniciando download para sua máquina local...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Download iniciado! Verifique sua pasta de Downloads.


In [ ]:
# 1. INSTALAÇÃO E IMPORTAÇÃO
!pip install joblib scikit-learn pandas numpy

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from google.colab import files
import joblib
import os

# --- CARREGAMENTO (Assumindo que você já subiu os arquivos) ---
path = "/content/"
orders = pd.read_csv(path + 'olist_orders_dataset.csv')
items = pd.read_csv(path + 'olist_order_items_dataset.csv')
customers = pd.read_csv(path + 'olist_customers_dataset.csv')
sellers = pd.read_csv(path + 'olist_sellers_dataset.csv')
products = pd.read_csv(path + 'olist_products_dataset.csv')
geolocation = pd.read_csv(path + 'olist_geolocation_dataset.csv')

print("⚙️ Processando Geolocalização e Joins...")

# 3.1. Agrupar Geo (Centróide do CEP)
geo_ref = geolocation.groupby('geolocation_zip_code_prefix')[['geolocation_lat', 'geolocation_lng']].mean().reset_index()

# 3.2. Unificar Tabelas
items_first = items.drop_duplicates('order_id', keep='first')
df = orders.merge(items_first, on='order_id', how='left')
df = df.merge(customers[['customer_id', 'customer_zip_code_prefix']], on='customer_id', how='left')
df = df.merge(sellers[['seller_id', 'seller_zip_code_prefix']], on='seller_id', how='left')
df = df.merge(products[['product_id', 'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm']], on='product_id', how='left')

# 3.3. Merge das Coordenadas
# Destino (Cliente)
df = df.merge(geo_ref, left_on='customer_zip_code_prefix', right_on='geolocation_zip_code_prefix', how='left')
df.rename(columns={'geolocation_lat': 'lat_cust', 'geolocation_lng': 'lng_cust'}, inplace=True)
df.drop('geolocation_zip_code_prefix', axis=1, inplace=True)

# Origem (Vendedor)
df = df.merge(geo_ref, left_on='seller_zip_code_prefix', right_on='geolocation_zip_code_prefix', how='left')
df.rename(columns={'geolocation_lat': 'lat_sell', 'geolocation_lng': 'lng_sell'}, inplace=True)
df.drop('geolocation_zip_code_prefix', axis=1, inplace=True)

print("🧠 Calculando Distância e Features...")

# 3.4. Limpeza e Target
df = df[df['order_status'] == 'delivered'].dropna(subset=['order_delivered_customer_date', 'lat_cust', 'lat_sell', 'product_weight_g'])
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])
df['order_delivered_customer_date'] = pd.to_datetime(df['order_delivered_customer_date'])

# --- ADIÇÃO: Calcular a Estimativa Antiga (Necessário para o gráfico) ---
df['order_estimated_delivery_date'] = pd.to_datetime(df['order_estimated_delivery_date'])
df['dias_estimados_antigo'] = (df['order_estimated_delivery_date'] - df['order_purchase_timestamp']).dt.days
# -----------------------------------------------------------------------

# Target: Dias Reais
df['dias_reais'] = (df['order_delivered_customer_date'] - df['order_purchase_timestamp']).dt.days
df = df[(df['dias_reais'] > 0) & (df['dias_reais'] < 60)]

# 3.5. Feature: Distância (Haversine)
def haversine_vec(lat1, lon1, lat2, lon2):
    R = 6371
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    dphi = np.radians(lat2 - lat1)
    dlambda = np.radians(lon2 - lon1)
    a = np.sin(dphi/2)**2 + np.cos(phi1)*np.cos(phi2)*np.sin(dlambda/2)**2
    return 2*R*np.arcsin(np.sqrt(a))

df['distancia_km'] = haversine_vec(df['lat_sell'], df['lng_sell'], df['lat_cust'], df['lng_cust'])
df['volume_cm3'] = df['product_length_cm'] * df['product_height_cm'] * df['product_width_cm']

features = ['distancia_km', 'product_weight_g', 'volume_cm3', 'freight_value', 'price']
X = df[features]
y = df['dias_reais']

print(f"🚀 Treinando modelo com {len(df)} registros...")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=100, max_depth=20, n_jobs=-1, random_state=42)
model.fit(X_train, y_train)

# Avaliação Rápida
mae = mean_absolute_error(y_test, model.predict(X_test))
print(f"📊 Sucesso! O modelo erra em média apenas {mae:.2f} dias.")

# --- ADIÇÃO: Gerar Dados para o Gráfico ---
print("📊 Gerando arquivo de comparação para o gráfico...")
# Usamos o modelo para prever todos os dados e comparar com a estimativa antiga
df['dias_previstos_ia'] = model.predict(X)
# Salvamos uma amostra de 5000 linhas para o arquivo não ficar gigante
df_final = df[['dias_reais', 'dias_estimados_antigo', 'dias_previstos_ia']].sample(min(5000, len(df)))
df_final.to_csv('comparativo_modelo.csv', index=False)
# ------------------------------------------

print("💾 Salvando arquivos...")

# Salvar no disco do Colab
joblib.dump(model, 'modelo_entregas.joblib')
geo_ref.to_csv('referencia_geo.csv', index=False)

print("⬇️ Iniciando download para sua máquina local...")
try:
    files.download('modelo_entregas.joblib')
    files.download('referencia_geo.csv')
    files.download('comparativo_modelo.csv') # Baixa o novo arquivo também!
    print("✅ Download iniciado! Verifique sua pasta de Downloads.")
except Exception as e:
    print("⚠️ O download automático falhou (comum em alguns navegadores).")
    print("👉 Por favor, vá na aba 'Arquivos' na esquerda, clique com botão direito nos arquivos gerados e escolha 'Baixar'.")

⚙️ Processando Geolocalização e Joins...
🧠 Calculando Distância e Features...
🚀 Treinando modelo com 95660 registros...
📊 Sucesso! O modelo erra em média apenas 5.00 dias.
📊 Gerando arquivo de comparação para o gráfico...
💾 Salvando arquivos...
⬇️ Iniciando download para sua máquina local...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Download iniciado! Verifique sua pasta de Downloads.


In [ ]:
print("🔄 Retreinando com filtro de 'Mundo Eficiente'...")

# 1. Carrega os dados (Assumindo que df já existe na memória do passo anterior)
# Se não estiver, recarregue os CSVs e faça os merges iniciais do script anterior.
# Vou assumir que 'df' já tem as colunas 'dias_reais', 'distancia_km', etc.

# --- O PULO DO GATO: LIMPEZA DE OUTLIERS ---
# Vamos manter apenas entregas que levaram até 20 dias (cortando a cauda longa de problemas)
# E vamos remover dados da época da Greve dos Caminhoneiros (Maio/2018)
df_clean = df.copy()

# 1. Filtro de Data (Remove Greve de Maio/2018)
df_clean = df_clean[~((df_clean['order_purchase_timestamp'].dt.year == 2018) & (df_clean['order_purchase_timestamp'].dt.month == 5))]

# 2. Filtro Estatístico (Mantém apenas 90% dos dados mais rápidos)
limite_aceitavel = df_clean['dias_reais'].quantile(0.90) # Pega o percentil 90
df_clean = df_clean[df_clean['dias_reais'] <= limite_aceitavel]

print(f"📉 Removidos {len(df) - len(df_clean)} pedidos problemáticos (Greves/Extravios).")
print(f"🚀 Novo teto de prazo considerado: {limite_aceitavel} dias.")

# --- TREINAMENTO ---
features = ['distancia_km', 'product_weight_g', 'volume_cm3', 'freight_value', 'price']
X = df_clean[features]
y = df_clean['dias_reais']

# Usando parâmetros mais robustos para evitar overfitting
model_opt = RandomForestRegressor(n_estimators=150, max_depth=15, min_samples_split=5, random_state=42, n_jobs=-1)
model_opt.fit(X, y)

# Salva o novo modelo por cima do antigo
joblib.dump(model_opt, 'modelo_entregas.joblib')

print("✅ Modelo Otimizado Salvo! Pode rodar o Streamlit novamente.")

🔄 Retreinando com filtro de 'Mundo Eficiente'...
📉 Removidos 14629 pedidos problemáticos (Greves/Extravios).
🚀 Novo teto de prazo considerado: 23.0 dias.
✅ Modelo Otimizado Salvo! Pode rodar o Streamlit novamente.
